# L2 Congestion Scenarios 

## Summary

The L2 Congestion scenario evaluates the relationship between network congestion measures and mana limits (target mana limit and hard mana limit per block) when different oracle control parameters are adjusted. This scenario aims to understand how these factors influence network performance and congestion levels.

## Scenario 1 

### Objective 

### Experimental Setup

#### Testing Variables: 

##### Environmental:

- Simulating transactions based on varying user maximum mana amounts
- (Potential) Congestion scenarios:  
    i) Varying sequencer profitability conditions  
    ii) Different oracle parameter values affecting transaction inclusion

##### Protocol: 

- Assess the impact of mana limits on network congestion. This scenario tests various parameter choices related to mana usage and oracle controls. Key parameters include:  
    i) TARGET_MANA_PER_BLOCK_C - The intended average mana usage per block  
    ii) MAXIMUM_MANA_PER_BLOCK_C - The hard limit on mana usage per block  
    iii) Oracle parameters affecting transaction costs and inclusion

#### Simulation input/output per Monte Carlo run:

##### Input:
- Resulting transactions from varying user maximum mana amounts, sequencer profitability conditions, and oracle parameter values. These inputs are used to assess the impact on network congestion.

##### Output:  
- Time series for finalized blocks/epochs, measured according to the level of congestion (i.e., the actually attained mana use).
- Monte Carlo M runs indexed by m:  
For each parameter constellation of interest, we draw M realizations of the sequencer's (check) maximum mana price they are willing to pay. Additionally, for each transaction, draw the maximum mana amount the transaction can use.

#### Sweep Parameters:

 Refer to the [spreadsheet](https://docs.google.com/spreadsheets/d/1EbW4sEYWb7iCjOYfgivNsLxe1O6ZFYwFSsRQjfiPLTM/edit?gid=955157985#gid=955157985&range=A1) for detailed parameter configurations related to staking and slashing mechanisms.

##### Control:

##### Environmental:

#### Simulation Behavior: 

#### Threshold Inequalities:

#### Metrics:

1. **Percentage Near Target:**
    
    - Compute the percentage of MC runs where the actual average mana used per block was within some percentage of the TARGET_MANA_PER_BLOCK_C control variable.
    - **Interpretation:** A low value implies that the system was not operating at its intended target, as mana used in a block was on average far above or far below the TARGET_MANA_PER_BLOCK_C value.
    
2. **Percentage Near Limit:**
    
    - Compute the percentage of MC runs where the actual average mana used per transaction was within some percentage of the MAXIMUM_MANA_PER_BLOCK_C control variable.
    - **Interpretation:** A high value implies that the system exhibited high congestion.

In [ ]:
#### Computational Complexity:
1. Total number of parameter constellations: :dart: 
2. Total number of Monte Carlo runs per constellation: :dart:
3. Total number of experiments per adaptive grid: :dart:
4. Number of adaptive grid searches: :dart:
5. Total number of parameter constellations evaluated: :dart:

### Protocol Parameter Recommendations 

:::info
See [main doc](https://hackmd.io/@blockscience/B1QKItvEye) for copyable explanations (or referenceable)
:::

- [ ] :dart: Write descriptive interpretation over table :dart: 
- [ ] :dart: Generate an analogue table below for the results :dart: 

### Decision Tree and Parameter Importance

:::info
See [main doc](https://hackmd.io/@blockscience/B1QKItvEye) for copyable explanations (or referenceable)
:::
- [ ] :dart: Write descriptive interpretation over the plot below :dart: 
- [ ] :dart: Generate an analogue plot below for the results :dart:

### Commentary on Results:

:dart: Write anything that comes up to mind based on the interpretation over all the results :dart: 

## Conclusion

- [ ] 🔫 Jakob owns after Danilo wrote commentary on results 🔫

## Appendix:
### Parameter Impact on Metrics: 